### Comparison

In [32]:
%%writefile test_.py
def plusone(x):
    return x+1
def test_one():
    assert plusone(3) == 5
def test_2():
    #comparison
    assert 'foo 1 bar' == 'foo 2 bar'

def test_3():
    #comparison2
    a = 'a'*100+'1'+'b'*100
    b = 'a'*100+'2'+'b'*100
    assert a == b

def test_4():
    #comparison2
    a = set([1,2,3])
    b = set([1,3,4])
    assert a == b
    
def test_5():
    #comparison2
    a = set([1,2,3])
    b = set([1,3,4])
    assert a == b
def test_6():
    a={'a':'b'}
    b={'x':'d'}
    assert a == b

Overwriting test_.py


### Test Discovery

In [33]:
!py.test 

============================= test session starts ==============================
platform darwin -- Python 2.7.10 -- py-1.4.25 -- pytest-2.6.3
collected 6 items 

test_.py FFFFFF

=================================== FAILURES ===================================
___________________________________ test_one ___________________________________

    def test_one():
>       assert plusone(3) == 5
E       assert 4 == 5
E        +  where 4 = plusone(3)

test_.py:4: AssertionError
____________________________________ test_2 ____________________________________

    def test_2():
        #comparison
>       assert 'foo 1 bar' == 'foo 2 bar'
E       assert 'foo 1 bar' == 'foo 2 bar'
E         - foo 1 bar
E         ?     ^
E         + foo 2 bar
E         ?     ^

test_.py:7: AssertionError
____________________________________ test_3 ____________________________________

    def test_3():
        #comparison2
        a = 'a'*100+'1'+'b'*100
        b = 'a'*100+'2'+'b'*100
>       assert a == b
E   

### Exception Handling

In [1]:
%%writefile test_exp.py
import pytest
def f():
    raise SystemExit(1)
def test_exception():
    with pytest.raises(SystemExit):
        f()

Overwriting test_exp.py


### Fixtures (Setup & Teardown)

@pytest.fixture(autouse=True) act as setup/teardown without explicit request by test function

In [46]:
%%writefile test_fixtures.py
import pytest

class Person:
    def greet(self):
        print "greet called ..."
        return "Hello World"
@pytest.fixture
def person():
    return Person()
def test_fix(person):
    greeting = person.greet()
    assert greeting == "Hi World"

Overwriting test_fixtures.py


### Verifying calls

Mock(spec=SomeClass) #spec => raise an exception if I'm accessing an attribute which is not there in 'SomeClass'

In [52]:
%%writefile mock_example.py
class DB:
    def __init__(self):
        pass
    def persist(self, person):
        pass
    
class Person:
    def __init__(self, db, name):
        self.db = db
        self.name = name
        
    def save(self):
        self.db.persist(self)
    

Writing mock_example.py


In [81]:
%%writefile test_mock.py
import pytest
from mock import MagicMock
from mock_example import Person, DB

@pytest.fixture
def mock_db():
    return MagicMock(spec=DB)

def test_save_persist_to_db(mock_db):
    sashi = Person("Sashi", mock_db)
    sashi.save()
    mock_db.persist.assert_called_with(sashi)
    
def test_save_persist_to_db(mock_db):
    mock_db.persist.assert_called_with()
    
def test_called_with_other_agr(mock_db):
    mock_db.persist(1,2,3)
    mock_db.persist.assert_called_with()
    
def test_any_call_mock_db(mock_db):
    mock_db.persist(1)
    mock_db.persist(3)
    #assert_called_with() will check for the last call
    mock_db.persist.assert_any_call(1)

Overwriting test_mock.py


In [ ]:
!py.test test_mock.py

In [76]:
%%writefile weather_service.py
import random

class WeatherService:
    def barometer(self):
        return random.choise(['rising','falling'])#Non deterministic

Overwriting test_mock.py


In [83]:
%%writefile forecastor.py

import random

class WeatherService:
    def barometer(self):
        return random.choise(['rising','falling'])#Non deterministic

class ForeCaster:
    def __init__(self, weatherservice):
        self.weatherservice = weatherservice
    
    def forecast(self):
        reading = self.weatherservice.barometer()
        forcast_dict = dict(
        rising ='Going to Rain',
        falling = 'Looks Clear'
        )
        return forcast_dict[reading]

Overwriting forecastor.py


In [90]:
%%writefile test_mock_ws.py
import pytest
from mock import MagicMock
from forecastor import ForeCaster, WeatherService

@pytest.fixture
def mock_ws():
    return MagicMock(spec=WeatherService)

def test_rain_when_barometer_rising(mock_ws):
    forecaster = ForeCaster(mock_ws)
    mock_ws.barometer.return_value = 'rising'
    assert forecaster.forecast() == 'Going to Rain'
#instead of two test cases for rising and falling
@pytest.mark.parametrize("reading, expected_forecast",
                         [('rising', 'Going to Rain'),
                          ('falling','Looks Clear')])
def test_forecast(reading, expected_forecast, mock_ws):
    forecaster = ForeCaster(mock_ws)
    mock_ws.barometer.return_value = reading
    assert forecaster.forecast() == expected_forecast
    
    
    

Overwriting test_mock_ws.py


In [91]:
!py.test -v test_mock_ws.py

============================= test session starts ==============================
platform darwin -- Python 2.7.10 -- py-1.4.25 -- pytest-2.6.3 -- //anaconda/bin/python
collected 3 items 

test_mock_ws.py::test_rain_when_barometer_rising PASSED
test_mock_ws.py::test_forecast[rising-Going to Rain] PASSED
test_mock_ws.py::test_forecast[falling-Looks Clear] PASSED

=========================== 3 passed in 0.02 seconds ===========================


### MonkeyPatching

Its a predefined fixture

use monkeypatch.setattr() for patch value

In [93]:
%%writefile forecastor2.py

import random

class WeatherService:
    def barometer(self):
        return random.choise(['rising','falling'])#Non deterministic

class ForeCaster:
    def __init__(self):
        self.weatherservice = WeatherService()
    
    def forecast(self):
        reading = self.weatherservice.barometer()
        forcast_dict = dict(
        rising ='Going to Rain',
        falling = 'Looks Clear'
        )
        return forcast_dict[reading]

Writing forecastor2.py


In [100]:
%%writefile test_mock_ws2.py
import pytest
from mock import MagicMock
from forecastor import ForeCaster, WeatherService

@pytest.fixture
def mock_ws():
    return MagicMock(spec=WeatherService)

def test_rain_when_barometer_rising(monkeypatch, mock_ws):
    WS = MagicMock(return_value=mock_ws)
    monkeypatch.setattr('forecastor.WeatherService',WS)
    forecaster = ForeCaster(mock_ws)
    mock_ws.barometer.return_value = 'rising'
    assert forecaster.forecast() == 'Going to Rain'
#instead of two test cases for rising and falling
@pytest.mark.parametrize("reading, expected_forecast",
                         [('rising', 'Going to Rain'),
                          ('falling','Look3s Clear')])
def test_forecast(reading, expected_forecast, monkeypatch, mock_ws):
    WS = MagicMock(return_value=mock_ws)
    monkeypatch.setattr('forecastor.WeatherService',WS)
    forecaster = ForeCaster(mock_ws)
    mock_ws.barometer.return_value = reading
    assert forecaster.forecast() == expected_forecast
    

Overwriting test_mock_ws2.py


In [ ]:
!py.test -v test_mock_ws2.py --ipdb

In [103]:
print "hello"

hello
